# Student Loan Risk with Deep Learning

In [204]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [205]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [206]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [207]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [208]:
# Define the target set y using the credit_ranking column
y=loans_df['credit_ranking']

# Display a sample of y
display(y.head())

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [209]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
display(X.head())

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [210]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [211]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled)

array([[-0.56794375, -1.18088385, -0.19241296, ...,  0.73249574,
        -0.22553894, -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, ..., -0.03016065,
        -0.75844691, -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021, ..., -1.36480934,
         0.42579302,  0.05358797],
       ...,
       [-0.9589175 ,  1.07974571, -0.89792713, ...,  0.22405814,
        -0.9952949 ,  0.33314696],
       [-0.62379715,  0.54947458, -1.35147195, ...,  0.35116754,
        -0.46238693, -1.34420697],
       [ 0.43741731, -0.73433974,  1.16822152, ..., -0.66570765,
         0.18894504,  1.91731456]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [212]:
# Define the the number of inputs (features) to the model

number_input_features = len(X.columns)

# Review the number of features
display(number_input_features)


11

In [213]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 8

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 5

# Define the number of neurons in the output layer
output_nodes = 1


In [214]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu', input_dim=number_input_features))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [215]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [216]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [217]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - accuracy: 0.5615 - loss: 0.6864
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - accuracy: 0.6111 - loss: 0.6646
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - accuracy: 0.6582 - loss: 0.6584
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step - accuracy: 0.6744 - loss: 0.6233
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - accuracy: 0.6825 - loss: 0.6159
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.6873 - loss: 0.6008
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step - accuracy: 0.6806 - loss: 0.5985
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7136 - loss: 0.5609
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 290us/step - accuracy: 0.7126 - loss: 0.5568
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step - accuracy: 0.7414 - loss: 0.5299
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 301us/step - accuracy: 0.7099 - loss: 0.5656
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [218]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 4ms/step - accuracy: 0.7400 - loss: 0.5196
Loss: 0.519616961479187, Accuracy: 0.7400000095367432


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [219]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [220]:
# Set the model's file path
file_path = Path('saved_models/student_loans.keras')

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [221]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 2ms/step


array([[0.6344148 ],
       [0.3356415 ],
       [0.71426135],
       [0.69902605],
       [0.9808044 ],
       [0.6969317 ],
       [0.89194804],
       [0.14664005],
       [0.72694904],
       [0.2533165 ],
       [0.9525379 ],
       [0.2604266 ],
       [0.38983884],
       [0.92962146],
       [0.53652227],
       [0.46946976],
       [0.89599526],
       [0.35268122],
       [0.62863547],
       [0.41552252],
       [0.4647005 ],
       [0.71573323],
       [0.291221  ],
       [0.9721656 ],
       [0.38213736],
       [0.9437893 ],
       [0.83999157],
       [0.57307124],
       [0.22899598],
       [0.4392091 ],
       [0.43820995],
       [0.93612367],
       [0.19342601],
       [0.93041754],
       [0.28506342],
       [0.6356317 ],
       [0.30128083],
       [0.3519746 ],
       [0.88157815],
       [0.18729828],
       [0.9273973 ],
       [0.07107324],
       [0.06102584],
       [0.9184302 ],
       [0.06232943],
       [0.6424535 ],
       [0.17391309],
       [0.284

In [222]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [223]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.70      0.77      0.74       188
           1       0.78      0.71      0.74       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.74      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

Response Notes:  
Looking at the data collected for a student loan recommendation engine we should be evaluating the success factors for being to repay a student loan.  This would include information such as the type of degree the student is working toward as well as overall CPA and past success for alumni with a the same degree. 

This is information is collected in the dataset provided and all can help to predict the financial success of a student and therefore the ability to repay loans.  For example, selection of a high demand degree will increase the likelihood of obtaining employement with in that field and financial success.  Information such as GPA can be an indicator of overall responsibility for a a student and looking at historical data for others in that field can indicate the liklihood of employement and ability to repay loans.

In this case all this type of data is relevant to the ability of a student to repay a loan

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Response Notes:

In the case of a student loan recommendation engine collaborative filtering may be the most useful option for evaluation.  In this case, we are looking at the historical success of other individuals with the same degree, similiar GPAs and prior success based on a major across many people to predict success of loan repayment.  Basically it is most effective to look at the data from individuals with similar backgrounds and compare that information to the loan in question. 

While content and context based filtering could also be predictive the history of others with simlilar characteristics seems to be the best predictive element. 

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
Response Notes:

Some real world challenges associated with a recommendation model for a student loan system could be as follows:

1.  The data based on the initial recommendation could change.  For example, a student may enter into a degree program for a STEM based options and later change to an alternate field with this less employement potential after starting their program.  This recommendation engine does not seem to account for the likelihood of changes in degree which could alter the potential employment paths for a student and therefore the potential to pay back the loan.  The change in student interest would not be accounted for in the algorithm.

2.  There may be a level of data sparseness accociated with a data set.  For example, this data does not appear to take into consideration the type of (or calliber of) the institution the student is attending.  This could impact the level of employement they are able to attain and therefore repayment rates.  Other examples of missing data that could be helpful would be options such as graduation rates in the degree program selected.  


